# DATA ANALYTICS: MODULO 3. Ejercicio de evaluación final

In [1]:
#IMPORTAR LIBRERIAS

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
#EXTRACCION DE LOS DATOS

df_flight = pd.read_csv('./datasets/Customer Flight Activity.csv')
df_loyalty = pd.read_csv('./datasets/Customer Loyalty History.csv')

In [ ]:
#EXPLORACION DE LOS DATOS

En primer lugar hago una primera exploración usando el head para ver como se ven los datos y el .info() para analizar si las columnas tienen el tipo de dato correcto. También reviso el nombre de las columnas -podria usar el .columns pero ya con el .info() lo estoy viendo-, por si tengo que cambiar o eliminar alguna

In [3]:
df_flight.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [4]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB


In [5]:
df_loyalty.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [ ]:
df_loyalty.info()

- En relación al tipo de dato parece que este todo correcto.
- En relación con los valores nulos:
    - Los valores nulos en Cancellation Year y Cancellation Month parecen coherentes, ya que probablemente corresponden a clientes que siguen activos en el programa de fidelización.
    - Los valores nulos de Salario deberan ser revisados

In [ ]:
#Voy a mirar para los datos del tipo "object" cuales son sus valores unicos

In [6]:
df_loyalty.describe(include = 'object').T

,count,unique,top,freq
Country,16737,1,Canada,16737
Province,16737,11,Ontario,5404
City,16737,29,Toronto,3351
Postal Code,16737,55,V6E 3D9,911
Gender,16737,2,Female,8410
Education,16737,5,Bachelor,10475
Marital Status,16737,3,Married,9735
Loyalty Card,16737,3,Star,7637
Enrollment Type,16737,2,Standard,15766


In [ ]:
# La columna Country, no aporta información adicional, dado que al revisarla, hemos comprobado que todos los datos tienen en la columna pais "canada"

In [7]:
print(f'Valores unicos de Province: {df_loyalty['Province'].unique()}')
print(f'Valores unicos de City: {df_loyalty['City'].unique()}')
print(f'Valores unicos de Education: {df_loyalty['Education'].unique()}')
print(f'Valores unicos de Gender: {df_loyalty['Gender'].unique()}')
print(f'Valores unicos de Loyalty Card: {df_loyalty['Loyalty Card'].unique()}')
print(f'Valores unicos de Enrollment Type: {df_loyalty['Enrollment Type'].unique()}')

Valores unicos de Province: ['Ontario' 'Alberta' 'British Columbia' 'Quebec' 'Yukon' 'New Brunswick'
 'Manitoba' 'Nova Scotia' 'Saskatchewan' 'Newfoundland'
 'Prince Edward Island']
Valores unicos de City: ['Toronto' 'Edmonton' 'Vancouver' 'Hull' 'Whitehorse' 'Trenton' 'Montreal'
 'Dawson Creek' 'Quebec City' 'Fredericton' 'Ottawa' 'Tremblant' 'Calgary'
 'Thunder Bay' 'Whistler' 'Peace River' 'Winnipeg' 'Sudbury'
 'West Vancouver' 'Halifax' 'London' 'Regina' 'Kelowna' "St. John's"
 'Victoria' 'Kingston' 'Banff' 'Moncton' 'Charlottetown']
Valores unicos de Education: ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
Valores unicos de Gender: ['Female' 'Male']
Valores unicos de Loyalty Card: ['Star' 'Aurora' 'Nova']
Valores unicos de Enrollment Type: ['Standard' '2018 Promotion']


In [8]:
for col in df_loyalty.select_dtypes(include='object'):
    print('-----------------------------')
    print(df_loyalty[col].unique())
    print(df_loyalty[col].value_counts())

-----------------------------
['Canada']
Country
Canada    16737
Name: count, dtype: int64
-----------------------------
['Ontario' 'Alberta' 'British Columbia' 'Quebec' 'Yukon' 'New Brunswick'
 'Manitoba' 'Nova Scotia' 'Saskatchewan' 'Newfoundland'
 'Prince Edward Island']
Province
Ontario                 5404
British Columbia        4409
Quebec                  3300
Alberta                  969
Manitoba                 658
New Brunswick            636
Nova Scotia              518
Saskatchewan             409
Newfoundland             258
Yukon                    110
Prince Edward Island      66
Name: count, dtype: int64
-----------------------------
['Toronto' 'Edmonton' 'Vancouver' 'Hull' 'Whitehorse' 'Trenton' 'Montreal'
 'Dawson Creek' 'Quebec City' 'Fredericton' 'Ottawa' 'Tremblant' 'Calgary'
 'Thunder Bay' 'Whistler' 'Peace River' 'Winnipeg' 'Sudbury'
 'West Vancouver' 'Halifax' 'London' 'Regina' 'Kelowna' "St. John's"
 'Victoria' 'Kingston' 'Banff' 'Moncton' 'Charlottetown']
Cit

CONCLUSIONES EXPLORACIÓN:
- ELIMINAR COLUMNA PAIS (Todos los datos son el mismo)

In [9]:
df_loyalty.drop(columns = ['Country'], inplace = True) 
df_loyalty.columns

Index(['Loyalty Number', 'Province', 'City', 'Postal Code', 'Gender',
       'Education', 'Salary', 'Marital Status', 'Loyalty Card', 'CLV',
       'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month'],
      dtype='object')

In [ ]:
#Ahora revisaremos si hay valores duplicados

In [10]:
df_flight.duplicated().sum()

1864

In [11]:
#ATENCION! tengo valores duplicados, voy a revisar si tiene sentido o deberia eliminarlos

df_flight[df_flight.duplicated(keep=False)].sort_values(by=['Loyalty Number', 'Year', 'Month'])

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
41,101902,2017,1,0,0,0,0,0.0,0,0
42,101902,2017,1,0,0,0,0,0.0,0,0
16942,101902,2017,2,0,0,0,0,0.0,0,0
16943,101902,2017,2,0,0,0,0,0.0,0,0
33843,101902,2017,3,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
151972,992168,2017,9,0,0,0,0,0.0,0,0
304080,992168,2018,6,0,0,0,0,0.0,0,0
304081,992168,2018,6,0,0,0,0,0.0,0,0
354783,992168,2018,9,0,0,0,0,0.0,0,0


Los registros duplicados detectados en Customer Flight Activity corresponden a filas completamente idénticas en todas las columnas, por lo que no aportan información adicional y pueden eliminarse sin pérdida de información.

In [12]:
#ELIMINAR VALORES DUPLICADOS
df_flight.drop_duplicates(inplace=True)

In [ ]:
#Seguimos analizando si hay otros duplicados, ya que los que he analizado ahora son que coincidan exactamente todas las lineas

In [13]:
df_flight.duplicated(subset=['Loyalty Number', 'Year', 'Month']).sum()

2072

In [14]:
df_flight[df_flight.duplicated(subset=['Loyalty Number', 'Year', 'Month'],keep=False)]

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
114,106001,2017,1,9,4,13,1365,136.0,0,0
115,106001,2017,1,5,0,5,910,91.0,0,0
275,114414,2017,1,8,4,12,3132,313.0,0,0
276,114414,2017,1,1,1,2,1094,109.0,0,0
488,126490,2017,1,6,0,6,1902,190.0,0,0
...,...,...,...,...,...,...,...,...,...,...
405299,981508,2018,12,5,5,10,4030,403.0,0,0
405451,990512,2018,12,15,5,20,2940,294.0,0,0
405452,990512,2018,12,5,2,7,3906,390.0,0,0
405486,992168,2018,12,15,0,15,3120,312.0,0,0


Aunque existen múltiples registros para un mismo cliente en un mismo mes, estos no corresponden a duplicados exactos, ya que presentan valores diferentes en variables clave como distancia, vuelos o puntos acumulados. Por tanto, estos registros se consideran observaciones válidas y no se eliminan.

In [15]:
df_loyalty.duplicated().sum()

0

In [16]:
df_loyalty.duplicated(subset=['Loyalty Number']).sum()

0

No se detectan valores duplicados en el conjunto de datos de historial de lealtad ni a nivel de filas completas ni a nivel del identificador único del cliente (Loyalty Number). Esto indica que cada cliente está representado una única vez, lo cual es coherente con la naturaleza del datase

In [ ]:
#REVISIÓN DE SI HAY VALORES ATIPICOS

In [17]:
df_flight.describe().T

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,403760.0,549875.383713,258961.514684,100018.0,326699.0,550598.0,772152.0,999986.0
Year,403760.0,2017.500352,0.500000,2017.0,2017.0,2018.0,2018.0,2018.0
Month,403760.0,6.501335,3.451982,1.0,4.0,7.0,10.0,12.0
Flights Booked,403760.0,4.134050,5.230064,0.0,0.0,1.0,8.0,21.0
Flights with Companions,403760.0,1.036569,2.080472,0.0,0.0,0.0,1.0,11.0
Total Flights,403760.0,5.170619,6.526858,0.0,0.0,1.0,10.0,32.0
Distance,403760.0,1214.460979,1434.098521,0.0,0.0,525.0,2342.0,6293.0
Points Accumulated,403760.0,124.263761,146.696179,0.0,0.0,53.0,240.0,676.5
Points Redeemed,403760.0,30.838587,125.758002,0.0,0.0,0.0,0.0,876.0
Dollar Cost Points Redeemed,403760.0,2.495973,10.172033,0.0,0.0,0.0,0.0,71.0


In [18]:
df_loyalty.describe().T

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,16737.0,549735.880445,258912.132453,100018.00,326603.00,550434.00,772019.00,999986.00
Salary,12499.0,79245.609409,35008.297285,-58486.00,59246.50,73455.00,88517.50,407228.00
CLV,16737.0,7988.896536,6860.982280,1898.01,3980.84,5780.18,8940.58,83325.38
Enrollment Year,16737.0,2015.253211,1.979111,2012.00,2014.00,2015.00,2017.00,2018.00
Enrollment Month,16737.0,6.669116,3.398958,1.00,4.00,7.00,10.00,12.00
Cancellation Year,2067.0,2016.503145,1.380743,2013.00,2016.00,2017.00,2018.00,2018.00
Cancellation Month,2067.0,6.962748,3.455297,1.00,4.00,7.00,10.00,12.00


In [ ]:
#SALARIOS EN NEGATIVO --> REVISAR

In [19]:
df_loyalty[df_loyalty["Salary"]<0]


,Loyalty Number,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
1082,542976,Quebec,Montreal,H2Y 4R4,Male,High School or Below,-49830.0,Divorced,Star,24127.50,2018 Promotion,2018,3,NaN,NaN
1894,959977,British Columbia,Vancouver,V5R 1W3,Female,Bachelor,-12497.0,Married,Aurora,9453.00,2018 Promotion,2018,3,NaN,NaN
2471,232755,British Columbia,Vancouver,V1E 4R6,Female,Bachelor,-46683.0,Single,Nova,4787.81,2018 Promotion,2018,3,NaN,NaN
3575,525245,British Columbia,Victoria,V10 6T5,Male,Bachelor,-45962.0,Married,Star,2402.33,2018 Promotion,2018,3,NaN,NaN
3932,603070,British Columbia,West Vancouver,V6V 8Z3,Female,Bachelor,-19325.0,Single,Star,2893.74,2018 Promotion,2018,3,NaN,NaN
4712,491242,British Columbia,Dawson Creek,U5I 4F1,Male,Bachelor,-43234.0,Married,Star,7597.91,2018 Promotion,2018,3,NaN,NaN
6560,115505,Newfoundland,St. John's,A1C 6H9,Male,Bachelor,-10605.0,Married,Nova,5860.17,2018 Promotion,2018,4,NaN,NaN
6570,430398,Newfoundland,St. John's,A1C 6H9,Male,Bachelor,-17534.0,Married,Nova,49423.80,2018 Promotion,2018,3,NaN,NaN
7373,152016,Ontario,Toronto,P1J 8T7,Female,Bachelor,-58486.0,Married,Aurora,5067.21,2018 Promotion,2018,2,NaN,NaN
8576,194065,Ontario,Sudbury,M5V 1G5,Female,Bachelor,-31911.0,Married,Nova,2888.85,2018 Promotion,2018,2,NaN,NaN


Los valores negativos detectados en la variable Salary no son coherentes con el contexto del análisis. 
Ante esta situación, existen distintas alternativas metodológicas:
- tratar dichos valores como datos nulos o 
- considerar su valor absoluto. 

Dado que no es posible conocer con certeza el origen del error y siguiendo el criterio indicado en clase, en este ejercicio se opta por utilizar el valor absoluto de los salarios para poder continuar con el análisis.

In [ ]:
#ANALISIS DE DATOS NULOS

In [ ]:
nulos = df_flight.isnull().sum()
nulos

In [ ]:
nulos = df_loyalty.isnull().sum()
nulos

In [ ]:
nulos = df_loyalty.isnull().sum()/df_loyalty.shape[0]*100
nulos

Tal como habiamos avanzado con anterioridad:
- Cancellation Year y Cancellation Month → nulos lógicos (clientes activos)
- Salary -> Nulos relevantes ya que los usaremos en analisis posteriores.

Para el "Salary", al tratarse de una variable numerica y de un porcentaje ALTO de nulos aplicaremos alguna técnica más avanzada de imputación. (Si fuera un porcentaje más bajo podriamos aplicar la media o mediana)

In [ ]:
## GESTION DE LOS NULOS 
## SALARY

imputer_knn = KNNImputer(n_neighbors=5)
df_loyalty['Salary'] = imputer_knn.fit_transform(df_loyalty[['Salary']])

#Me decanto por imputer KNNI dado que el dataset no es muy grande


In [ ]:
nulos = df_loyalty.isnull().sum()/df_loyalty.shape[0]*100
nulos

In [ ]:
sns.boxplot(x=df_flight['Flights Booked'])
plt.title('Boxplot – Flights Booked')
plt.show()

In [ ]:
sns.boxplot(x=df_flight['Distance'])
plt.title('Boxplot – Distance')
plt.show()

In [ ]:
sns.boxplot(x=df_flight['Points Accumulated'])
plt.title('Boxplot – Points Accumulated')
plt.show()

In [ ]:
sns.boxplot(x=df_loyalty['Salary'])
plt.title('Boxplot – Salary')
plt.show()

In [ ]:
sns.boxplot(x=df_loyalty['CLV'])
plt.title('Boxplot – CLV')
plt.show()

In [ ]:
df_flight['Flights Booked'].hist(bins=30)
plt.title('Histogram – Flights Booked')
plt.xlabel('Flights Booked')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df_flight['Distance'].hist(bins=30)
plt.title('Histogram – Distance')
plt.xlabel('Distance')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df_flight['Points Accumulated'].hist(bins=30)
plt.title('Histogram – Points Accumulated')
plt.xlabel('Points Accumulated')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df_loyalty['Salary'].hist(bins=30)
plt.title('Histogram – Salary')
plt.xlabel('Salary')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df_loyalty['CLV'].hist(bins=30)
plt.title('Histogram – CLV')
plt.xlabel('CLV')
plt.ylabel('Frequency')
plt.show()